# Dependencies

In [1]:
import os
import random
import mne
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy.signal as signal
%matplotlib qt

from sklearn.metrics import mean_squared_error, accuracy_score, precision_score
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from skimage.metrics import structural_similarity as ssim
from sklearn.model_selection import train_test_split

import torch
import snntorch as snn
from snntorch import spikegen
from snntorch import surrogate
from snntorch import utils
import snntorch.functional as SF
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import logging
import warnings
warnings.filterwarnings("ignore")
logging.getLogger('mne').setLevel(logging.WARNING)

# Directories

In [2]:
main_folder = "ThesisFolder"
extras_folder = "Extras"

data_folder_py = os.path.join(extras_folder, "GAMEEMO_PY")
data_folder_mat = os.path.join(extras_folder,"GAMEEMO_MAT")
data_folder_fif = os.path.join(main_folder,"GAMEEMO_FIF")
data_folder_epoch = os.path.join(extras_folder,"GAMEEMO_EPOCH")

files_py = os.listdir(data_folder_py)
files_mat = os.listdir(data_folder_mat)
files_fif = os.listdir(data_folder_fif)

filename = os.path.join(data_folder_fif, files_fif[0])
raw = mne.io.read_raw_fif(filename, preload=True)
info = raw.info
channels = raw.info["ch_names"]
print(info)
print(channels)

<Info | 10 non-empty values
 bads: []
 ch_names: AF3, F7, F3, FC5, T7, P7, O1, O2, P8, T8, FC6, F4, F8, AF4
 chs: 14 EEG
 custom_ref_applied: False
 dig: 17 items (3 Cardinal, 14 EEG)
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 lowpass: 64.0 Hz
 meas_date: unspecified
 meas_id: 4 items (dict)
 nchan: 14
 projs: []
 sfreq: 128.0 Hz
>
['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']


# Function Definition space

In [6]:
def a_law_encode(x, A):
    abs_x = np.abs(x)
    encoded = np.where(abs_x < 1/A, A * abs_x / (1 + np.log(A)), (1 + np.log(A * abs_x)) / (1 + np.log(A)))
    return np.sign(x) * encoded

def a_law_decode(y, A):
    abs_y = np.abs(y)
    decoded = np.where(abs_y < 1 / (1 + np.log(A)), abs_y * (1 + np.log(A)) / A, np.exp(abs_y * (1 + np.log(A)) - 1))
    return np.sign(y) * decoded

In [7]:
def butterworth_bandpass_filter(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a

def plot_frequency_response(b, a, fs, nfft=8000, window='hamming'):
    w, h = signal.freqz(b, a, worN=nfft)
    plt.figure()
    plt.plot(0.5 * fs * w / np.pi, np.abs(h), 'b')
    plt.title('Bandpass Filter Frequency Response')
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Gain')
    plt.grid()
    plt.show()

def apply_filter(data, b, a):
    y = signal.filtfilt(b, a, data)
    return y

## Particle Swarm Optimization - Shuvankar 

In [5]:
def BSA_encode(S, thresholdBSA):
    B = (S > thresholdBSA).astype(int)  
    S_modified = S * B
    return B, S_modified

In [6]:
def calculate_fitness(original_signal, reconstructed_signal):
    mse = mean_squared_error(original_signal, reconstructed_signal)
    rmse = np.sqrt(mse)
    pcc, _ = pearsonr(original_signal, reconstructed_signal)
    snr = 10 * np.log10(np.sum(np.square(original_signal)) / np.sum(np.square(original_signal - reconstructed_signal)))
    psnr = 10 * np.log10(np.max(np.square(original_signal)) / mse)
    cos_sim = cosine_similarity([original_signal], [reconstructed_signal])[0][0]
    ssim_value = ssim(original_signal, reconstructed_signal, data_range=original_signal.max() - original_signal.min())
    
    # Combine all metrics for the final fitness score
    # Minimize MSE and RMSE, maximize SNR, PSNR, PCC, Cosine Similarity, and SSIM
    fitness = -mse + -rmse + (snr * 10) + (psnr * 5) + (pcc * 100) + (cos_sim * 100) + (ssim_value * 100)
    return fitness

In [7]:
def initialize_particles(size):
    particles = []
    for _ in range(size):
        # Randomly initialize thresholds for BSA
        position = (random.uniform(0.4, 0.7),)  # Threshold BSA
        velocity = (random.uniform(-0.1, 0.1),)  # Velocity for BSA threshold
        particles.append((position, velocity, position))  # (position, velocity, personal best)
    return particles

In [8]:
def particle_swarm_optimization(df, num_particles, num_iterations):
    inertia_weight = 0.7
    cognitive_weight = 1.5
    social_weight = 1.5
    
    particles = initialize_particles(num_particles)
    global_best_position = None
    global_best_fitness = -np.inf

    for iteration in range(num_iterations):
        for i in range(num_particles):
            position, velocity, personal_best = particles[i]

            # Assume we're processing the first channel for simplicity
            eeg_channel_data = df

            # Apply BSA and reconstruct
            spike_train_bsa, modified_signal_bsa = BSA_encode(eeg_channel_data, position[0])
            reconstructed_bsa = modified_signal_bsa

            # Calculate fitness
            fitness_bsa = calculate_fitness(eeg_channel_data, reconstructed_bsa)

            # Update personal best
            if fitness_bsa > calculate_fitness(eeg_channel_data, BSA_encode(eeg_channel_data, personal_best[0])[1]):
                personal_best = position

            # Update global best
            if fitness_bsa > global_best_fitness:
                global_best_fitness = fitness_bsa
                global_best_position = position

            # Update velocity and position
            new_velocity = (
                inertia_weight * velocity[0] +
                cognitive_weight * random.random() * (personal_best[0] - position[0]) +
                social_weight * random.random() * (global_best_position[0] - position[0]),
            )
            new_position = (min(max(position[0] + new_velocity[0], 0.0), 1.0),)  # Clamp BSA threshold

            particles[i] = (new_position, new_velocity, personal_best)

    return global_best_position, global_best_fitness

In [9]:
def optimize_eeg_signal(signal, num_particles=100, num_iterations=20):
    df = signal
    global channels

    global_best_position, global_best_fitness = particle_swarm_optimization(df, num_particles, num_iterations)
    return global_best_position[0], global_best_fitness

In [10]:
def generate_spikes(df, threshold):
    spikes_df = []
    for channel in df:
        eeg_channel_data = channel
        spike_train_bsa, _ = BSA_encode(eeg_channel_data, threshold)
        spikes_df.append(spike_train_bsa)
    return np.array(spikes_df)

# Preprocessing Zone

## Filtering Segment

## Filter Everything

In [8]:
rawArray = []
compressedRaw = []
for file in tqdm(files_fif):
    filename = os.path.join(data_folder_fif, file)
    raw = mne.io.read_raw_fif(filename, preload=True)
    raw.filter(8, 42) 

    epoch_length = 10 
    sfreq = raw.info['sfreq']

    events = mne.make_fixed_length_events(raw, duration=epoch_length)
    epochs = mne.Epochs(raw, events, tmin=0, tmax=epoch_length, baseline=None, preload=True)

    raw_data = epochs.get_data() * 1e6
    compressed_raw = np.zeros_like(raw_data)
    
    for epoch in raw_data:
        rawEpoch = []
        compressedEpoch = []
        for channel in epoch:
            c_ch = channel/np.max(channel)
            c_ch = a_law_encode(c_ch, 80)
            c_ch = (c_ch-np.min(c_ch))/(np.max(c_ch)-np.min(c_ch))
            channel = (channel-np.min(channel))/(np.max(channel)-np.min(channel))
            rawEpoch.append(channel)
            compressedEpoch.append(c_ch)
        rawArray.append(rawEpoch)
        compressedRaw.append(compressedEpoch)
        

rawArray =np.array(rawArray, dtype=np.float32)
compressedRaw =np.array(compressedRaw, dtype=np.float32)

print(sfreq)
print("Files: ", rawArray.shape[0], "\n",
     "Channels: ", rawArray.shape[1], "\n",
     "Samples: ", rawArray.shape[2], "\n",)

print("Files: ", compressedRaw.shape[0], "\n",
     "Channels: ", compressedRaw.shape[1], "\n",
     "Samples: ", compressedRaw.shape[2], "\n",)

100%|████████████████████████████████████████████████████████████████████████████████| 112/112 [00:21<00:00,  5.24it/s]


128.0
Files:  3248 
 Channels:  14 
 Samples:  1281 

Files:  3248 
 Channels:  14 
 Samples:  1281 



In [14]:
files = raw_data.shape[0]
print(raw_data.shape)

(29, 14, 1281)


In [10]:
# lowcut = 6.0 
# highcut = 12.0  
# fs = 128.0  
order = 2  

b_alpha, a_alpha = butterworth_bandpass_filter(8, 13, 128, order)
b_beta, a_beta   = butterworth_bandpass_filter(13, 30, 128, order)
b_gamma, a_gamma = butterworth_bandpass_filter(30, 60, 128, order)

In [11]:
alpha_band_raw = []
beta_band_raw = []
gamma_band_raw = []

for epoch in tqdm(rawArray):
    epochArrayA = []
    epochArrayB = []
    epochArrayG = []
    for channel in epoch:
        data_alpha = apply_filter(channel, b_alpha, a_alpha)
        data_alpha = ((data_alpha-np.min(data_alpha))/(np.max(data_alpha)-np.min(data_alpha)))*(np.max(channel))
        data_beta = apply_filter(channel, b_beta, a_beta)
        data_beta = ((data_beta-np.min(data_beta))/(np.max(data_beta)-np.min(data_beta)))*(np.max(channel))
        data_gamma = apply_filter(channel, b_gamma, a_gamma)
        data_gamma = ((data_gamma-np.min(data_gamma))/(np.max(data_gamma)-np.min(data_gamma)))*(np.max(channel))
        epochArrayA.append(data_alpha)
        epochArrayB.append(data_beta)
        epochArrayG.append(data_gamma)
    alpha_band_raw.append(epochArrayA)
    beta_band_raw.append(epochArrayB)
    gamma_band_raw.append(epochArrayG)


alpha_band_compressed = []
beta_band_compressed = []
gamma_band_compressed = []

for epoch in tqdm(compressedRaw):
    epochArrayA = []
    epochArrayB = []
    epochArrayG = []
    for channel in epoch:
        data_alpha = apply_filter(channel, b_alpha, a_alpha)
        data_alpha = ((data_alpha-np.min(data_alpha))/(np.max(data_alpha)-np.min(data_alpha)))*(np.max(channel))
        data_beta = apply_filter(channel, b_beta, a_beta)
        data_beta = ((data_beta-np.min(data_beta))/(np.max(data_beta)-np.min(data_beta)))*(np.max(channel))
        data_gamma = apply_filter(channel, b_gamma, a_gamma)
        data_gamma = ((data_gamma-np.min(data_gamma))/(np.max(data_gamma)-np.min(data_gamma)))*(np.max(channel))
        epochArrayA.append(data_alpha)
        epochArrayB.append(data_beta)
        epochArrayG.append(data_gamma)
    alpha_band_compressed.append(epochArrayA)
    beta_band_compressed.append(epochArrayB)
    gamma_band_compressed.append(epochArrayG)

alpha_band_raw = np.array(alpha_band_raw)
beta_band_raw = np.array(beta_band_raw)
gamma_band_raw = np.array(gamma_band_raw)
alpha_band_compressed = np.array(alpha_band_compressed)
beta_band_compressed = np.array(beta_band_compressed)
gamma_band_compressed = np.array(gamma_band_compressed)

100%|█████████████████████████████████████████████████████████████████████████████| 3248/3248 [00:21<00:00, 147.96it/s]


In [12]:
print("Files: ", alpha_band_raw.shape[0], "\n",
     "Channels: ", alpha_band_raw.shape[1], "\n",
     "Samples: ", alpha_band_raw.shape[2], "\n",)

print("Files: ", alpha_band_compressed.shape[0], "\n",
     "Channels: ", alpha_band_compressed.shape[1], "\n",
     "Samples: ", alpha_band_compressed.shape[2], "\n",)

Files:  3248 
 Channels:  14 
 Samples:  1281 

Files:  3248 
 Channels:  14 
 Samples:  1281 



In [15]:
offset = 48
multiplier = 95
epoch = (files * multiplier) + offset
channel = 7
sample_data = rawArray[epoch][channel]
compressed_sample = compressedRaw[epoch][channel]

sample_data_alpha = alpha_band_raw[epoch][channel]
compressed_sample_alpha = alpha_band_compressed[epoch][channel]

sample_data_beta = beta_band_raw[epoch][channel]
compressed_sample_beta = beta_band_compressed[epoch][channel]

sample_data_gamma = gamma_band_raw[epoch][channel]
compressed_sample_gamma = gamma_band_compressed[epoch][channel]

plt.figure(2)
plt.subplot(421)
plt.plot(sample_data)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Raw Array file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.subplot(422)
plt.plot(compressed_sample)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Compressed Array file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

# plt.figure()
plt.subplot(423)
plt.plot(sample_data_alpha)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Raw Array Alpha file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.subplot(424)
plt.plot(compressed_sample_alpha)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Compressed Array Alpha file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

# plt.figure()
plt.subplot(425)
plt.plot(sample_data_beta)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Raw Array Beta file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.subplot(426)
plt.plot(compressed_sample_beta)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Compressed Array Beta file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

# plt.figure()
plt.subplot(427)
plt.plot(sample_data_gamma)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Raw Array Gamma file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.subplot(428)
plt.plot(compressed_sample_gamma)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Compressed Array Gamma file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.tight_layout()
plt.show()

In [2]:
rawArray_loaded = np.load(os.path.join("HolyMotherOfDataset5sec", "rawArray.npy"))
compressedRaw_loaded = np.load(os.path.join("HolyMotherOfDataset5sec", "compressedRaw.npy"))
alpha_band_raw_loaded = np.load(os.path.join("HolyMotherOfDataset5sec", "alpha_band_raw.npy"))
beta_band_raw_loaded = np.load(os.path.join("HolyMotherOfDataset5sec", "beta_band_raw.npy"))
gamma_band_raw_loaded = np.load(os.path.join("HolyMotherOfDataset5sec", "gamma_band_raw.npy"))
alpha_band_compressed_loaded = np.load(os.path.join("HolyMotherOfDataset5sec", "alpha_band_compressed.npy"))
beta_band_compressed_loaded = np.load(os.path.join("HolyMotherOfDataset5sec", "beta_band_compressed.npy"))
gamma_band_compressed_loaded = np.load(os.path.join("HolyMotherOfDataset5sec", "gamma_band_compressed.npy"))

In [3]:
offset = 48
multiplier = 48
epoch = (59 * multiplier) + offset
channel = 7
sample_data = rawArray_loaded[epoch][channel]
compressed_sample = compressedRaw_loaded[epoch][channel]

print("Raw\n Files: ", rawArray_loaded.shape[0], "\n",
     "Channels: ", rawArray_loaded.shape[1], "\n",
     "Samples: ", rawArray_loaded.shape[2], "\n",)

print("Compressed\n Files: ", compressedRaw_loaded.shape[0], "\n",
     "Channels: ", compressedRaw_loaded.shape[1], "\n",
     "Samples: ", compressedRaw_loaded.shape[2], "\n",)

sample_data_alpha = alpha_band_raw_loaded[epoch][channel]
compressed_sample_alpha = alpha_band_compressed_loaded[epoch][channel]

print("Raw Alpha\n Files: ", alpha_band_raw_loaded.shape[0], "\n",
     "Channels: ", alpha_band_raw_loaded.shape[1], "\n",
     "Samples: ", alpha_band_raw_loaded.shape[2], "\n",)

print("Compressed Alpha\n Files: ", alpha_band_compressed_loaded.shape[0], "\n",
     "Channels: ", alpha_band_compressed_loaded.shape[1], "\n",
     "Samples: ", alpha_band_compressed_loaded.shape[2], "\n",)

sample_data_beta = beta_band_raw_loaded[epoch][channel]
compressed_sample_beta = beta_band_compressed_loaded[epoch][channel]

print("Raw Beta\n Files: ", beta_band_raw_loaded.shape[0], "\n",
     "Channels: ", beta_band_raw_loaded.shape[1], "\n",
     "Samples: ", beta_band_raw_loaded.shape[2], "\n",)

print("Compressed Beta\n Files: ", beta_band_compressed_loaded.shape[0], "\n",
     "Channels: ", beta_band_compressed_loaded.shape[1], "\n",
     "Samples: ", beta_band_compressed_loaded.shape[2], "\n",)

sample_data_gamma = gamma_band_raw_loaded[epoch][channel]
compressed_sample_gamma = gamma_band_compressed_loaded[epoch][channel]

print("Raw Gamma\n Files: ", gamma_band_raw_loaded.shape[0], "\n",
     "Channels: ", gamma_band_raw_loaded.shape[1], "\n",
     "Samples: ", gamma_band_raw_loaded.shape[2], "\n",)

print("Compressed Gamma\n Files: ", gamma_band_compressed_loaded.shape[0], "\n",
     "Channels: ", gamma_band_compressed_loaded.shape[1], "\n",
     "Samples: ", gamma_band_compressed_loaded.shape[2], "\n",)

plt.figure(3)
plt.subplot(421)
plt.plot(sample_data)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Raw Array file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.subplot(422)
plt.plot(compressed_sample)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Compressed Array file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

# plt.figure()
plt.subplot(423)
plt.plot(sample_data_alpha)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Raw Array Alpha file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.subplot(424)
plt.plot(compressed_sample_alpha)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Compressed Array Alpha file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

# plt.figure()
plt.subplot(425)
plt.plot(sample_data_beta)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Raw Array Beta file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.subplot(426)
plt.plot(compressed_sample_beta)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Compressed Array Beta file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

# plt.figure()
plt.subplot(427)
plt.plot(sample_data_gamma)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Raw Array Gamma file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.subplot(428)
plt.plot(compressed_sample_gamma)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Compressed Array Gamma file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.tight_layout()
plt.show()

Raw
 Files:  6608 
 Channels:  14 
 Samples:  641 

Compressed
 Files:  6608 
 Channels:  14 
 Samples:  641 

Raw Alpha
 Files:  6608 
 Channels:  14 
 Samples:  641 

Compressed Alpha
 Files:  6608 
 Channels:  14 
 Samples:  641 

Raw Beta
 Files:  6608 
 Channels:  14 
 Samples:  641 

Compressed Beta
 Files:  6608 
 Channels:  14 
 Samples:  641 

Raw Gamma
 Files:  6608 
 Channels:  14 
 Samples:  641 

Compressed Gamma
 Files:  6608 
 Channels:  14 
 Samples:  641 



In [22]:
rawArray_loaded = np.load(os.path.join("HolyMotherOfDataset10sec", "rawArray.npy"))
compressedRaw_loaded = np.load(os.path.join("HolyMotherOfDataset10sec", "compressedRaw.npy"))
alpha_band_raw_loaded = np.load(os.path.join("HolyMotherOfDataset10sec", "alpha_band_raw.npy"))
beta_band_raw_loaded = np.load(os.path.join("HolyMotherOfDataset10sec", "beta_band_raw.npy"))
gamma_band_raw_loaded = np.load(os.path.join("HolyMotherOfDataset10sec", "gamma_band_raw.npy"))
alpha_band_compressed_loaded = np.load(os.path.join("HolyMotherOfDataset10sec", "alpha_band_compressed.npy"))
beta_band_compressed_loaded = np.load(os.path.join("HolyMotherOfDataset10sec", "beta_band_compressed.npy"))
gamma_band_compressed_loaded = np.load(os.path.join("HolyMotherOfDataset10sec", "gamma_band_compressed.npy"))

In [23]:
offset = 23
multiplier = 48
epoch = (29 * multiplier) + offset
channel = 7
sample_data = rawArray_loaded[epoch][channel]
compressed_sample = compressedRaw_loaded[epoch][channel]

print("Raw\n Files: ", rawArray_loaded.shape[0], "\n",
     "Channels: ", rawArray_loaded.shape[1], "\n",
     "Samples: ", rawArray_loaded.shape[2], "\n",)

print("Compressed\n Files: ", compressedRaw_loaded.shape[0], "\n",
     "Channels: ", compressedRaw_loaded.shape[1], "\n",
     "Samples: ", compressedRaw_loaded.shape[2], "\n",)

sample_data_alpha = alpha_band_raw_loaded[epoch][channel]
compressed_sample_alpha = alpha_band_compressed_loaded[epoch][channel]

print("Raw Alpha\n Files: ", alpha_band_raw_loaded.shape[0], "\n",
     "Channels: ", alpha_band_raw_loaded.shape[1], "\n",
     "Samples: ", alpha_band_raw_loaded.shape[2], "\n",)

print("Compressed Alpha\n Files: ", alpha_band_compressed_loaded.shape[0], "\n",
     "Channels: ", alpha_band_compressed_loaded.shape[1], "\n",
     "Samples: ", alpha_band_compressed_loaded.shape[2], "\n",)

sample_data_beta = beta_band_raw_loaded[epoch][channel]
compressed_sample_beta = beta_band_compressed_loaded[epoch][channel]

print("Raw Beta\n Files: ", beta_band_raw_loaded.shape[0], "\n",
     "Channels: ", beta_band_raw_loaded.shape[1], "\n",
     "Samples: ", beta_band_raw_loaded.shape[2], "\n",)

print("Compressed Beta\n Files: ", beta_band_compressed_loaded.shape[0], "\n",
     "Channels: ", beta_band_compressed_loaded.shape[1], "\n",
     "Samples: ", beta_band_compressed_loaded.shape[2], "\n",)

sample_data_gamma = gamma_band_raw_loaded[epoch][channel]
compressed_sample_gamma = gamma_band_compressed_loaded[epoch][channel]

print("Raw Gamma\n Files: ", gamma_band_raw_loaded.shape[0], "\n",
     "Channels: ", gamma_band_raw_loaded.shape[1], "\n",
     "Samples: ", gamma_band_raw_loaded.shape[2], "\n",)

print("Compressed Gamma\n Files: ", gamma_band_compressed_loaded.shape[0], "\n",
     "Channels: ", gamma_band_compressed_loaded.shape[1], "\n",
     "Samples: ", gamma_band_compressed_loaded.shape[2], "\n",)

plt.figure(3)
plt.subplot(421)
plt.plot(sample_data)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Raw Array file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.subplot(422)
plt.plot(compressed_sample)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Compressed Array file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

# plt.figure()
plt.subplot(423)
plt.plot(sample_data_alpha)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Raw Array Alpha file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.subplot(424)
plt.plot(compressed_sample_alpha)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Compressed Array Alpha file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

# plt.figure()
plt.subplot(425)
plt.plot(sample_data_beta)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Raw Array Beta file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.subplot(426)
plt.plot(compressed_sample_beta)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Compressed Array Beta file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

# plt.figure()
plt.subplot(427)
plt.plot(sample_data_gamma)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Raw Array Gamma file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.subplot(428)
plt.plot(compressed_sample_gamma)
plt.xlabel("samples (n)")
plt.ylabel("values")
plt.title(f"Compressed Array Gamma file: {multiplier+1}, Channel: {channel+1}, segment: {offset+1}")

plt.tight_layout()
plt.show()

Raw
 Files:  3248 
 Channels:  14 
 Samples:  1281 

Compressed
 Files:  3248 
 Channels:  14 
 Samples:  1281 

Raw Alpha
 Files:  3248 
 Channels:  14 
 Samples:  1281 

Compressed Alpha
 Files:  3248 
 Channels:  14 
 Samples:  1281 

Raw Beta
 Files:  3248 
 Channels:  14 
 Samples:  1281 

Compressed Beta
 Files:  3248 
 Channels:  14 
 Samples:  1281 

Raw Gamma
 Files:  3248 
 Channels:  14 
 Samples:  1281 

Compressed Gamma
 Files:  3248 
 Channels:  14 
 Samples:  1281 



# Encoder workspace

In [3]:
class ThresholdBasedRepresentation:
    def __init__(self):
        self.threshold = 0
        self.signal = []

    def generate_EEG_TBR(self, dataArray):
        if len(dataArray.shape) > 1:
            print("Invalid Data Dimension. Expected 1D Array")
            return np.zeros_like(dataArray)
        dataArray_t = np.concatenate((dataArray, [dataArray[-1]]))
        dataArray_tminus1 = np.concatenate(([dataArray[-1]], dataArray))
        spike_train = dataArray_t - dataArray_tminus1
        spike_train = spike_train[0:-1]
        self.threshold = np.mean(spike_train) + 0.5 * np.std(spike_train)
        spike_train = np.where(spike_train > self.threshold, 1, 0)
        return spike_train, self.threshold

    def generate_EEG_SF(self, dataArray, threshold):
        if len(dataArray.shape) > 1:
            print("Invalid Data Dimension. Expected 1D Array")
            return np.zeros_like(dataArray)
        self.threshold = threshold
        baseline = dataArray[0]
        out = np.zeros_like(dataArray)
        bases = np.zeros_like(dataArray)
        for i, s_t in enumerate(dataArray):
            if s_t >= baseline + self.threshold:
                out[i] = 1
                bases[i] = baseline
                baseline += self.threshold
            elif s_t < baseline - self.threshold:
                out[i] = -1
                bases[i] = baseline
                baseline -= self.threshold
        return out, bases

    def generate_EEG_MW(self, dataArray, n_window, threshold):
        if len(dataArray.shape) > 1:
            print("Invalid Data Dimension. Expected 1D Array")
            return np.zeros_like(dataArray)

        L_original = len(dataArray)
        dataArray = np.concatenate(
            (dataArray, np.zeros(
                np.mod(L_original, n_window)
            ))
        )
        L = len(dataArray)
        self.threshold = threshold
        spike_train = np.zeros_like(dataArray)
        
        for window in np.arange(int(np.divide(L, n_window)-1)):
            dataChunk = dataArray[(window)*n_window:(window+1)*(n_window)]
            baseline = np.mean(dataChunk)
            for i, s_t in enumerate(dataChunk):
                if s_t >= baseline + self.threshold:
                    spike_train[(window)*n_window + i] = 1
                    baseline += self.threshold
                elif s_t < baseline - self.threshold:
                    spike_train[(window)*n_window + i] = -1
                    baseline -= self.threshold

        return spike_train[0:L_original]

    def generate_EEG_BSA(self, signal, fir, threshold):
        self.signal = np.copy(signal)
        if len(self.signal.shape) > 1:
            print("Invalid Data Dimension. Expected 1D Array")
            return np.zeros_like(self.signal)   
        L = len(self.signal)
        F = len(fir)
        spike_train = np.zeros(L)
        self.threshold = threshold
        for t in range(L - F):
            err1 = 0
            err2 = 0
            for k in range(F):
                err1 += abs(self.signal[t + k] - fir[k])
                err2 += abs(self.signal[t + k - 1])
            if err1 <= (err2 * self.threshold):
                spike_train[t] = 1
                for k in range(F):
                    self.signal[t + k + 1] -= fir[k]     
        return spike_train
    def apply_butterworth_filter(self, data, lowcut, highcut, fs, order=5):
        nyquist = 0.5 * fs
        low = lowcut / nyquist
        high = highcut / nyquist
        b, a = signal.butter(order, [low, high], btype='band')
        y = signal.filtfilt(b, a, data)
        return y 
        
    def decode_time_contrast(self, spike_train, threshold, encode_type):
        self.threshold = threshold
        recon = np.zeros_like(spike_train, dtype=np.float16)
        recon[-1] = self.threshold
        print(threshold)

        if encode_type=='tbr':
            recon = self.apply_butterworth_filter(spike_train, 1, 20, 128, 2)
        else:
            for idx, spike in enumerate(np.copy(spike_train)):
                if spike == 1:
                    recon[idx] = recon[idx-1] + threshold
                elif spike == -1:
                    recon[idx] = recon[idx-1] - threshold
                else:
                    recon[idx] = recon[idx-1]

        return recon

    def decode_BSA(self, spike_train, fir):
        recon = np.convolve(spike_train, fir, mode="same")
        return recon

In [27]:
tbr_encoder = ThresholdBasedRepresentation()
encoding_types = ['tbr', 'sf', 'mw', 'bsa']
current_encode = 'bsa'
# spike_data_gamma, threshold = tbr_encoder.generate_EEG_TBR(compressed_sample_alpha)
# spike_data_gamma, threshold = tbr_encoder.generate_EEG_SF(compressed_sample_alpha, 0.05)
# spike_data_gamma = tbr_encoder.generate_EEG_MW(compressed_sample_alpha, 16, 0.05)
# fir = (np.array([16, 8, 4, 2], dtype=np.float16))/32
# spike_data_gamma = tbr_encoder.generate_EEG_BSA(compressed_sample_alpha, fir , 0.7)

spike_read_dir = "HolyMotherOfDataset5sec"
read_filenames = ["rawArray.npy",
            "compressedRaw.npy",
            "alpha_band_raw.npy",
            "beta_band_raw.npy",
            "gamma_band_raw.npy",
            "alpha_band_compressed.npy",
            "beta_band_compressed.npy",
            "gamma_band_compressed.npy",]
spike_save_dir = "HolySpikes5s"+current_encode
os.makedirs(spike_save_dir, exist_ok=True)
filenames = ["rawArraySpikes.npy",
            "compressedRawSpikes.npy",
            "alpha_band_rawSpikes.npy",
            "beta_band_rawSpikes.npy",
            "gamma_band_rawSpikes.npy",
            "alpha_band_compressedSpikes.npy",
            "beta_band_compressedSpikes.npy",
            "gamma_band_compressedSpikes.npy",]

for fileID, filename in enumerate(read_filenames):
    filepath = os.path.join(spike_read_dir, filename)
    load_data = np.load(filepath)
    save_data = np.zeros_like(load_data)
    for idx, example in tqdm(enumerate(load_data)):
        ch_data = np.zeros_like(example)
        for i, channel in enumerate(example):
            if current_encode=='tbr':
                ch_data[i, :], _ = tbr_encoder.generate_EEG_TBR(channel)
            elif current_encode == 'sf':
                ch_data[i, :], _ = tbr_encoder.generate_EEG_SF(channel, 0.05)
            elif current_encode == 'mw':
                ch_data[i, :] = tbr_encoder.generate_EEG_MW(channel, 16, 0.05)
            elif current_encode == 'bsa':
                fir = (np.array([16, 8, 4, 2], dtype=np.float16))/32
                ch_data[i, :] = tbr_encoder.generate_EEG_BSA(channel, fir , 0.7)
        save_data[idx,:,:] = ch_data
    save_filepath = os.path.join(spike_save_dir, filenames[fileID])
    np.save(save_filepath, save_data)

6608it [08:33, 12.87it/s]
6608it [08:24, 13.10it/s]
6608it [04:15, 25.89it/s]
6608it [04:21, 25.25it/s]
6608it [04:15, 25.91it/s]
6608it [04:10, 26.42it/s]
6608it [04:17, 25.66it/s]
6608it [04:15, 25.86it/s]


In [111]:
encoding_types = ['tbr', 'sf', 'mw', 'bsa']
current_encoder = 'tbr'

read_dir = "HolyMotherOfDataset5sec"
read_filenames = ["rawArray.npy",
            "compressedRaw.npy",
            "alpha_band_raw.npy",
            "beta_band_raw.npy",
            "gamma_band_raw.npy",
            "alpha_band_compressed.npy",
            "beta_band_compressed.npy",
            "gamma_band_compressed.npy",]

type_ = 0
load_waveform = np.load(os.path.join(read_dir, read_filenames[type_]))
conv1 = nn.Conv1d(1, 1, kernel_size=15, padding=7, bias=True)

mx = 1 * int(len(load_waveform) / 112)
offset = 54
channel = 4
section = np.arange(600)+20
plt.subplot(511)
plt.plot(conv1(torch.tensor(load_waveform[mx + offset][channel][section], dtype=torch.float).unsqueeze(0)).squeeze(0).detach().cpu().numpy())
plt.plot(load_waveform[mx + offset][channel][section])
for i in np.arange(4):
    spike_dir = "HolySpikes5s"+encoding_types[i]
    filenames = ["rawArraySpikes.npy",
                "compressedRawSpikes.npy",
                "alpha_band_rawSpikes.npy",
                "beta_band_rawSpikes.npy",
                "gamma_band_rawSpikes.npy",
                "alpha_band_compressedSpikes.npy",
                "beta_band_compressedSpikes.npy",
                "gamma_band_compressedSpikes.npy",]
    load_spike = np.load(os.path.join(spike_dir, filenames[type_]))
    plt.subplot(5, 1, i+2)
    plt.stairs(load_spike[mx + offset][channel][section])
    if encoding_types[i]==current_encoder:
        print(f"{current_encoder} in Use")

tbr in Use


In [101]:
plt.plot(torch.tensor(load_waveform[mx + offset][channel][section], dtype=torch.float).detach().numpy())

## Relevant codes 

## Relevant codes

# NorseCoding

In [2]:
read_dir = "HolyMotherOfDataset5sec"
read_filenames = ["rawArray.npy",
            "compressedRaw.npy",
            "alpha_band_raw.npy",
            "beta_band_raw.npy",
            "gamma_band_raw.npy",
            "alpha_band_compressed.npy",
            "beta_band_compressed.npy",
            "gamma_band_compressed.npy",]

current_encoder = 'bsa'
spike_dir = "HolySpikes5s"+current_encoder
filenames = ["rawArraySpikes.npy",
            "compressedRawSpikes.npy",
            "alpha_band_rawSpikes.npy",
            "beta_band_rawSpikes.npy",
            "gamma_band_rawSpikes.npy",
            "alpha_band_compressedSpikes.npy",
            "beta_band_compressedSpikes.npy",
            "gamma_band_compressedSpikes.npy",]

type_ = 0
load_waveform = np.load(os.path.join(read_dir, read_filenames[type_]))
load_spike = np.load(os.path.join(spike_dir, filenames[type_]))
print(load_waveform.shape, load_spike.shape)

(6608, 14, 641) (6608, 14, 641)


In [3]:
label = ['boring', 'horror', 'calm', 'funny']
labels = []
for i in np.arange(28):
    labels.extend(label)

labels = np.array(labels)
print(labels.shape)
hmm, counts = np.unique(labels, return_counts=True)
print(hmm, counts)
main_labels = np.repeat(labels, 59)
hmm, counts = np.unique(main_labels, return_counts=True)
print(main_labels.shape, hmm, counts)

(112,)
['boring' 'calm' 'funny' 'horror'] [28 28 28 28]
(6608,) ['boring' 'calm' 'funny' 'horror'] [1652 1652 1652 1652]


In [4]:
translate = {'boring':0, 'horror':1, 'calm':2, 'funny':3}
print(translate.get(main_labels[60]))

main_labels = np.array([translate.get(label) for label in main_labels])
print(main_labels)

print(main_labels.shape)
print(np.unique(main_labels, return_counts=True))

num_classes = np.max(main_labels) + 1
labels = np.eye(num_classes)[main_labels]

print(labels.shape)
print(labels)

1
[0 0 0 ... 3 3 3]
(6608,)
(array([0, 1, 2, 3]), array([1652, 1652, 1652, 1652], dtype=int64))
(6608, 4)
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(load_waveform[:4000, :, :], main_labels[:4000], test_size=0.25, random_state=42)

print(X_train.shape)

(3000, 14, 641)


In [6]:
# Convert to PyTorch tensors
dtype_1 = torch.float32
dtype_2 = torch.long
num_steps = 32
X_train = torch.tensor(X_train, dtype=dtype_1, requires_grad=False).to("cuda")
X_test = torch.tensor(X_test, dtype=dtype_1, requires_grad=False).to("cuda")
y_train = torch.tensor(y_train, dtype=dtype_2, requires_grad=False).to("cuda")
y_test = torch.tensor(y_test, dtype=dtype_2, requires_grad=False).to("cuda")

In [7]:
# Create PyTorch DataLoader for batching
train_dataset = TensorDataset(X_train.unsqueeze(1), y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

test_dataset = TensorDataset(X_test.unsqueeze(1), y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=True)

In [8]:
print(X_train.shape)
print(y_train.shape)

torch.Size([3000, 14, 641])
torch.Size([3000])


In [9]:
class MyEEGSNNModel(nn.Module):
    def __init__(self, n_outputs):
        super().__init__()
        
        self.n_outputs = n_outputs

        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = (1, 7), padding = (0, 3), bias=True)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = (1, 7), padding = (0, 3), bias=True)
        self.lif1 = snn.Leaky(beta=0.9, threshold=1.0, spike_grad=snn.surrogate.sigmoid(), init_hidden=False, learn_beta=True, learn_threshold=True)
        self.dense1 = nn.Linear(32*14*641, self.n_outputs)
        self.lif2 = snn.Leaky(beta=0.9, threshold=1.0, spike_grad=snn.surrogate.sigmoid(), init_hidden=False, learn_beta=True, learn_threshold=True)
        # self.softmax = nn.Softmax(dim=1)
       
    def forward(self, x):
        self.n_samples = x.shape[3]
        self.n_channel = x.shape[2]
        self.batch_size = x.shape[0]
        
        mem1 = self.lif1.init_leaky() 
        mem2 = self.lif2.init_leaky()
        
        x = self.conv1(x)
        x = self.relu(x)
        x, mem1 = self.lif1(x, mem1) 
        x = self.conv2(x)
        x = self.dense1(x.view(self.batch_size, -1))
        x, mem2 = self.lif2(x, mem2)
           
        return x, mem2

In [10]:
def forward_pass(model, timesteps, data):
    spk_rec = []
    mem_rec = []
    utils.reset(model)

    for i in range(timesteps):
        spk_out, mem_out = model(data)
        spk_rec.append(spk_out)
        mem_rec.append(mem_out)

    return torch.stack(spk_rec).to("cuda"), torch.stack(mem_rec).to("cuda")
    

In [11]:
model = MyEEGSNNModel(4).to("cuda")
criterion = SF.ce_rate_loss()
loss = torch.zeros((1), dtype=torch.float32, device="cuda")

for inputs, targets in train_loader:
    ouput, mem_rec = forward_pass(model, 32, inputs)
    print(inputs.shape)
    print(ouput.shape)
    print(ouput.size(-1))
    print(mem_rec.shape)
    print(mem_rec.size(-1))

    loss = criterion(mem_rec, targets)
    print(loss.item())
    break

torch.Size([64, 1, 14, 641])
torch.Size([32, 64, 4])
4
torch.Size([32, 64, 4])
4
1.3810564279556274


In [12]:
torch.cuda.empty_cache()

In [13]:
model = MyEEGSNNModel(4).to("cuda")
criterion = SF.ce_rate_loss()
n_epochs = 10
learning_rate = 5e-2

model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))

for epoch in range(n_epochs):
    epoch_loss = 0.0

    for inputs, targets in tqdm(train_loader):
        optimizer.zero_grad()  

        outputs, mem_rec = forward_pass(model, 32, inputs)
        # loss = criterion(mem_rec, targets)
        loss = criterion(outputs, targets)
        loss.backward()         
        optimizer.step()  
        epoch_loss += loss.item() 

    print(f'Epoch [{epoch + 1}/{n_epochs}], Loss: {epoch_loss / len(train_loader):.4f}')

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [06:12<00:00,  8.09s/it]


Epoch [1/10], Loss: 1.3923


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [06:11<00:00,  8.08s/it]


Epoch [2/10], Loss: 1.3863


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [06:11<00:00,  8.08s/it]


Epoch [3/10], Loss: 1.3863


 89%|█████████████████████████████████████████████████████████████████████████         | 41/46 [05:39<00:41,  8.28s/it]


KeyboardInterrupt: 

In [ ]:
def evaluate_model(model, test_data):
    test_data = test_data.permute(1, 2, 3, 0).unsqueeze(1)

    model.eval()
    with torch.no_grad():
        output, mem_rec = forward_pass(model, test_data)
            
    return output, mem_rec

In [ ]:
X_test = X_test.permute(1, 2, 3, 0).unsqueeze(1)
spk_out, _ = evaluate_model(model, X_test[:200].squeeze(1).permute(3, 0, 1, 2))
accuracy = SF.acc.accuracy_rate(spk_out, y_test[:200])

In [19]:
print(accuracy)

0.295


In [ ]:
lsm = nn.LogSoftmax()
y_pred = lsm(torch.sum(evaluate_model(model, X_test), axis=0))
print(y_pred.shape)

In [ ]:
print(y_test[0], torch.argmax(y_pred[0]))

In [ ]:
print(torch.argmax(y_pred, axis=1).shape)
print(y_test.shape)

In [ ]:
y_p = torch.argmax(y_pred, axis=1).cpu().detach().numpy()
y_t = y_test.cpu().detach().numpy()
print(accuracy_score(y_p, y_t))
accuratey = np.sum(y_p == y_t) / len(y_t)
print(accuratey)

In [ ]:
lsm = nn.LogSoftmax()
y_pred = lsm(torch.sum(evaluate_model(model, X_train), axis=0))
print(y_pred.shape)
print(torch.argmax(y_pred, axis=1).shape)
print(y_train.shape)
y_p = torch.argmax(y_pred, axis=1).cpu().detach().numpy()
y_t = y_train.cpu().detach().numpy()
print(accuracy_score(y_p, y_t))

In [ ]:
torch.save(model, "SNNEEG_model.pth")

In [22]:
output_size = 4         
model = MyEEGSNNModel(output_size).to("cuda")

In [11]:
n_epochs = 1
learning_rate = 0.001

criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = ce_rate_loss()

train_model(model, train_loader, criterion, optimizer, n_epochs)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:24<00:00, 10.21s/it]

Epoch [1/1], Loss: 1.4960


In [ ]:
torch.save(model, "SNNEEG_model.pth")

In [12]:
torch.cuda.empty_cache()

In [91]:
output = evaluate_model(model, X_train)
y_pred = np.argmax(output.cpu().detach().numpy(), axis=1)
y_true = np.argmax(y_train.cpu().detach().numpy(), axis=1)
print(accuracy_score(y_pred, y_true))
torch.cuda.empty_cache()

OutOfMemoryError: CUDA out of memory. Tried to allocate 10.94 GiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 18.83 GiB is allocated by PyTorch, and 904.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [93]:
output = evaluate_model(model, X_test)
y_pred = np.argmax(output.cpu().detach().numpy(), axis=1)
y_true = np.argmax(y_test.cpu().detach().numpy(), axis=1)
print(accuracy_score(y_pred, y_true))
torch.cuda.empty_cache()

0.27382753403933435
